In [48]:
from causalml.dataset import synthetic_data
from causalml.inference.meta import BaseSRegressor, BaseTRegressor, BaseXRegressor, BaseRRegressor, LRSRegressor
import pandas as pd

# Read in Ace data
ace_data = pd.read_csv('ace_data.csv')

# y, X, treatment, tau, b ,e = synthetic_data(mode=1, n=10000, p=8, sigma=1.0)

In [49]:
# Get the column names of ace_data
column_names = ace_data.columns
print(column_names)

Index(['GENHLTH', 'MARITAL', '_SEX', 'MENTHLTH', '_EDUCAG', '_INCOMG1',
       'POORHLTH', 'ADDEPEV3', '_AGEG5YR', '_AGE65YR', '_AGE80', '_AGE_G',
       'DECIDE', 'DIFFALON', 'ACEDEPRS', 'ACEDRINK', 'ACEDRUGS', 'ACEPRISN',
       'ACEDIVRC', 'ACEPUNCH', 'ACEHURT1', 'ACESWEAR', 'ACETOUCH', 'ACETTHEM',
       'ACEHVSEX'],
      dtype='object')


Simple Example with Drugs as Treatment and Mental Health as Response

In [46]:
# filter out rows that have `nan` values in the 'ACEDRUGS' or 'MENTHLTH' columns
ace_data = ace_data.dropna(subset=['ACEDRUGS', 'MENTHLTH'])

# Declare the treatment and outcome columns
treatment = ace_data['ACEDRUGS']
y = ace_data['MENTHLTH']

# Subtract 1 from the treatment column
treatment = treatment - 1

print(treatment.unique())

# Declare X
X = ace_data[['_AGE_G', '_AGEG5YR']]

# # Convert the treatment and outcome columns to numpy arrays of shape (n, 1)
# treatment = treatment.values.reshape(-1, 1)
# y = y.values.reshape(-1, 1)
# X = X.values.reshape(-1, 1)

# # ravel all the arrays
# treatment = treatment.ravel()
# y = y.ravel()
# X = X.ravel()

print(len(X))
print(len(treatment))
print(len(y))

print(X.shape)
print(treatment.shape)
print(y.shape)

[1. 0. 8. 6.]
60802
60802
60802
(60802, 2)
(60802,)
(60802,)


In [47]:
learner_s = LRSRegressor()
ate_s = learner_s.estimate_ate(X=X, treatment=treatment, y=y)
print(ate_s)

(array([11.68550479,  5.05376888, 13.69543695]), array([10.69037366,  1.24334966, 11.06873388]), array([12.68063591,  8.8641881 , 16.32214002]))


In [3]:
cate_s = learner_s.fit_predict(X=X, treatment=treatment, y=y)
print(cate_s)


[[0.70202296]
 [0.70202296]
 [0.70202296]
 ...
 [0.70202296]
 [0.70202296]
 [0.70202296]]


10000